# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: *When the output of the prediction is continuous,it's a regression problem.When the output of the prediction is  discrete  ,it's a classification problem. In the students intervention problem,the students need to be separated into two groups,one group need intervention and the other don't. Apparantly,it's a classification problem and a binary problem. *

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [12]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [13]:
# TODO: Calculate number of students
n_students = np.shape(student_data)[0]

# TODO: Calculate number of features
n_features = np.shape(student_data)[1]-1

# TODO: Calculate passing students
n_passed = np.shape(student_data[ "passed"][student_data[ "passed"]=='yes'])[0]

# TODO: Calculate failing students
n_failed = np.shape(student_data[ "passed"][student_data[ "passed"]=='no'])[0]

# TODO: Calculate graduation rate
from __future__ import division
grad_rate = n_passed/n_students*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [14]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [15]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [16]:
# TODO: Import any additional functionality you may need here

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
np.random.shuffle([X_all,y_all])
X_train = X_all[0:num_train]
X_test = X_all[num_train:]
y_train = y_all[0:num_train]
y_test = y_all[num_train:]

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: 

*1-KNeighborsClassifier()

    1.1 KNN classifier can be used in visual tracking problem to differentiate the target and the background.
    references：New Video Target Tracking Algorithm Based on KNN 
    
    1.2 the strengths of the model：
     1)the model is simple,
     2)no need to train the model
     3)it performs well in high dimensional spaces or the number of samples is not too large
    
    1.3 the weaknesses of the model：
    1)it's kind of lazy learning,testing needs more time and space
    2)it performs poorly when the number of samples is too large
    
    1.4 why do I choose this model:
    Because in this problem,the number of samples is just several hundred and  there are many features.I believe it performs well.
    
    
 2-svm.SVC()

    2.1 SVM can be used in medical science to predict the patient get what kind of disease 
    references：SVM Based Follow-up System for Brain Tumor Evolution from Magnetic Resonance Images
    
    2.2 the strengths of the model：
     1) Effective in high dimensional spaces.
     2) Still effective in cases where number of dimensions is greater than the number of samples.
     3) Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
     4) Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.
   
    2.3 the weaknesses of the model：
     1) If the number of features is much greater than the number of samples,the method is likely to give poor performances.
     
    2.4 why do I choose this model:
    Because in this problem,the number of samples is greater than the number of features.
    
    
3-linear_model.LogisticRegression()


    3.1 LogisticRegression can also be used in medical science.
    references：Logic Regression for Provider Effects on Kidney Cancer Treatment Delivery
    3.2 the strengths of the model：
     1) the model is simple.
   
    3.3 the weaknesses of the model：
     1) it performs poorly when the number of features is too much.
     2) the feature which is non-linear need to be transformed
     
    3.4 why do I choose this model:
    Because in this problem,the number of samples and the number of features are not too many.
*

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [17]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [18]:
# TODO: Import the three supervised learning models from sklearn

from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import linear_model
# TODO: Initialize the three models

clf_A = KNeighborsClassifier()
clf_B = svm.SVC(random_state=1)
clf_C = linear_model.LogisticRegression(random_state=1)

# TODO: Set up the training set sizes
# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C]:
    for size in [100, 200, 300]:
        train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)
        print '\n'


Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0050 seconds.
F1 score for training set: 0.8750.
Made predictions in 0.0030 seconds.
F1 score for test set: 0.6815.


Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0020 seconds
Made predictions in 0.0060 seconds.
F1 score for training set: 0.8600.
Made predictions in 0.0080 seconds.
F1 score for test set: 0.7761.


Training a KNeighborsClassifier using a training set size of 300. . .
Trained model in 0.0030 seconds
Made predictions in 0.0160 seconds.
F1 score for training set: 0.8597.
Made predictions in 0.0050 seconds.
F1 score for test set: 0.7597.


Training a SVC using a training set size of 100. . .
Trained model in 0.0040 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8772.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7975.


Training a SVC using a training set size of 200. 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - KNeighborsClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0020            |      0.0030            |   0.8750         |   0.6815        |
| 200               |        0.0010           |         0.0040         |      0.8600      |    0.7761       |
| 300               |        0.0020           |       0.0060           |      0.8597      |   0.7597        |

** Classifer 2 - Support Vector Machines**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0030              |       0.0020          |   0.8772         |    0.7975       |
| 200               |     0.0090              |        0.0040          |   0.8474         |    0.8026       |
| 300               |      0.0150             |       0.0030           |    0.8686        |     0.7571      |

** Classifer 3 - LogisticRegression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0020              |      0.0000           |       0.9200     |    0.6950       |
| 200               |     0.0050              |      0.0000            |         0.8414   |      0.7154     |
| 300               |     0.0060              |      0.0000          |        0.8486    |    0.6552       |


## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: 
*I choose SVM as the best model.From the table above, we can see that SVM is more expensive than the other two models. KNN needs very little time in training period, LogicRegression needs very little time in predicting period. SVM gets the high F1 score on both train and test data. LogicRegression gets very high F1 score on train data but much lower on the test data.KNN preforms good when the train data is big enough but performs bad when the train data is small.
*

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: *Support Vector Machine : 

*We have red balls and blue balls and some of them are covered by cloth on the table. We need to guess which color the covered ball are. We see that red balls and blue balls are not mixed ,it seems that they can be separated into two parts,so we put a stick on the table and change its position untill one side are all red balls and the other side are all blue balls, we like red color and also like blue color, so I don't want to treat them unequality ,so we go on change the position of the stick to have as big a gap on either side of the stick as possible. Now we guess the covered balls' colors are just like the ones on the same side of the stick.  
   What if the balls are not on the table but floating in the air like balloons? We use papers to separate them. In 2 dimention space like table plane, we use 1 dimention line to separate, In 3 dimention space, we use 2 dimention plane to separate, So in N dimention space, we use N-1 dimention superplane.
   What if the balls on the table are just like the figure shows below, they can not be separated by a stick unless the stick is plastic and you bend it to a curve. We can record the balls' position ,calculate their center point, then move them to new positions based on their distance between the center. Now we can separate them and predict the covered ball's color.


In [19]:
import matplotlib  
import matplotlib.pyplot as plt
import random
import numpy as np
plt.figure(figsize=(8, 8), dpi=80)  
axes=plt.subplot(111)  
  
red_x = []
red_y = []
blue_x = []
blue_y = []
red_num = 0
blue_num = 0
red_r = []
blue_r = []
for i in range(500):
    x = random.uniform(-3,3) 
    y = random.uniform(-3,3)
    r = x*x + y*y
    if r > 6 and r < 7 and red_num <20:
        red_x.append(x)
        red_y.append(y)
        red_r.append(r)
        red_num += 1
    elif r <2 and blue_num<10:
        blue_x.append(x)
        blue_y.append(y)
        blue_r.append(r)
        blue_num +=1
red = axes.scatter(red_x, red_y, s=100, c='red')
blue = axes.scatter(blue_x, blue_y, s=100, c='blue')
plt.show()

plt.figure(figsize=(8, 8), dpi=80)  
axes=plt.subplot(111) 
red_new = axes.scatter(red_r, red_y, s=100, c='red')
blue_new = axes.scatter(blue_r, blue_y, s=100, c='blue')
plt.show()

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [20]:
# TODO: Import 'GridSearchCV' and 'make_scorer'

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import fbeta_score,make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit

# TODO: Create the parameters list you wish to tune
parameters = {'gamma' : [0.001,0.1,0.5,1]}

# TODO: Initialize the classifier
clf = svm.SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(fbeta_score, beta=1,pos_label=None, average='weighted')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
sss = StratifiedShuffleSplit(y_train, n_iter=10, test_size=0.1)

grid_obj = GridSearchCV(svm.SVC(), param_grid = parameters, scoring=f1_scorer,cv=sss)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0110 seconds.
Tuned model has a training F1 score of 0.9710.
Made predictions in 0.0050 seconds.
Tuned model has a testing F1 score of 0.8205.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: *Tuned model has a training F1 score of 0.9710.Tuned model has a testing F1 score of 0.8205.They are both higher than the untuned model.*

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.